## 🔰PyTorchでニューラルネットワーク基礎 #22.5【MultiHeadAttention】

* 次回（注意行列・attention weightsの可視化）の準備編
* attention weightsの抽出方法や計算を見ることが主目的

## MultiHeadAttention

### 論文（Attention Is All You Need）流
論文では、MultiHeadAttentionの出力を次の手順で説明している。
1. 分散表現行列$X$に$W^Q, W^K, W^V$行列を掛けて クエリ、キー、バリュー行列$Q, K, V$ を計算する。
    * $Q = XW^Q$
    * $K = XW^K$
    * $V = XW^V$

2. $Q, K, V$に $h$（ヘッドの数）個の行列$W_i^Q, W_i^K, W_i^V$を掛け算して、$Q_i, K_i, V_i$を用意する。
    * $Q_i = QW_i^Q$
    * $K_i = KW_i^K$
    * $V_i = VW_i^V$

3. それぞれの$Q_i, K_i, V_i$でattentionの計算を行う。
    ```math
    head_i = \text{Attention}(Q_i, K_i, V_i) = \text{softmax}(\frac{Q_i K_i^T}{\sqrt{d}}) V_i 
    ```

4. ヘッドを集めて、$W^O$を掛け算し、$X$と同じ形状で出力する。
    ```math
    \text{MultiHead}(Q,K,V) = \text{Concat}(head_1,…,head_h) W^O
    ```


Q, K, Vに各ヘッドごとに異なる行列を用いて Qi, Ki, Vi を作り、それぞれで注意機構を計算してから concat する形になっている。

### PyTorch流

PyTorch の実装では、効率的に実装している。
1. $W^Q, W^K, W^V$ を縦に結合したような行列$W$を分散表現行列$X$に掛けて $Q, K, V$ をまとめて計算
2. $Q, K, V$をヘッド数に等分割して $Q_i, K_i, V_i$ に対応
3. 分割された値でヘッドごとの注意機構を計算し $Attention(Q_i, K_i, V_i)$
4. 各ヘッドの出力を concatして、$Concat(head_1,...,head_h)W^O$を出力とする

### 確認したいこと
* PyTorchでattention weightsを取得する方法は？【1】
* 分割された$Q_1$、$K_1$などはどれ？【2】
* ヘッド毎の注意行列・attention weightsはどれ？【3】

### PyTorchのMultiheadAttention

* Documents: https://docs.pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html

* 【1】mhaのオプションで、need_weights=True（デフォルト値）、average_attn_weights=False を指定すると、ヘッド毎の重みが取得できる。
* 【3】mhaの出力にヘッド毎の重みがあらわれる

In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(55) # torch.randnをランダムにしたい場合は外す

seq_len = 5
embed_dim = 4
num_heads = 2

# ダミー入力 (batch, seq_len, embed_dim)
X = torch.randn(1, seq_len, embed_dim)

(1)
mha = nn.MultiheadAttention(embed_dim, num_heads, bias=False, batch_first=True)
mha.eval() # 評価モードにしてdropoutなどのランダム性を排除

# (2)
torch_output, torch_w = mha(query=X, key=X, value=X, average_attn_weights=False)

In [2]:
print(f"torch_output:\n{torch_output}")
print(f"attention weights:\n{torch_w}")

torch_output:
tensor([[[ 0.3529,  0.0220,  0.0969, -0.1303],
         [ 0.4565,  0.1399,  0.0720,  0.1694],
         [ 0.3354,  0.1571,  0.0336,  0.2145],
         [ 0.3725,  0.0816,  0.1403, -0.0746],
         [ 0.3248,  0.0932,  0.0436,  0.0879]]], grad_fn=<TransposeBackward0>)
attention weights:
tensor([[[[0.0424, 0.2048, 0.3446, 0.2414, 0.1667],
          [0.1729, 0.1644, 0.2146, 0.2448, 0.2033],
          [0.2667, 0.1591, 0.1675, 0.2068, 0.2000],
          [0.0698, 0.2457, 0.3001, 0.2061, 0.1782],
          [0.1792, 0.1710, 0.2114, 0.2354, 0.2030]],

         [[0.1456, 0.1290, 0.2313, 0.2845, 0.2096],
          [0.2896, 0.3155, 0.1334, 0.0994, 0.1622],
          [0.2136, 0.3166, 0.1714, 0.1335, 0.1649],
          [0.1254, 0.2581, 0.2383, 0.2125, 0.1655],
          [0.1764, 0.2372, 0.2087, 0.1930, 0.1846]]]], grad_fn=<ViewBackward0>)


## MultiheadAttentionの計算途中を表示してみる
mhaで利用した内部パラメータを取得する

In [3]:
W = mha.in_proj_weight      # (3*embed_dim, embed_dim) W_Q, W_K, W_Vを合わせた大きな行列

E = mha.embed_dim           # mhaで利用した分散表現の次元 4
H = mha.num_heads           # mhaで利用したヘッド数 2
D = E // H                  # ヘッド数に分割した時の分散表現の次元 2 

# W = [W_Q, W_K, W_V]という大きな行列（実際は縦だけど）
W_q = W[:E]
W_k = W[E:2*E]
W_v = W[2*E:]

In [4]:
print(f"QKVを作るWの形状：{W.shape}")
print(f"W_qの形状: {W_q.shape}")
print(f"W_kの形状: {W_k.shape}")
print(f"W_vの形状: {W_v.shape}")

QKVを作るWの形状：torch.Size([12, 4])
W_qの形状: torch.Size([4, 4])
W_kの形状: torch.Size([4, 4])
W_vの形状: torch.Size([4, 4])


In [5]:
torch.set_printoptions(linewidth=200)

QKV = F.linear(X, W)
print(f"QKV shape: {QKV.shape}")
print(f"QKV:\n{QKV}")

QKV shape: torch.Size([1, 5, 12])
QKV:
tensor([[[-1.3839,  0.3560, -0.5477,  0.5145,  1.5560, -0.1749,  1.3026, -0.2896,  1.4396, -0.2397,  0.6415,  1.2935],
         [-0.3053, -0.4555,  0.9167, -0.7092,  0.2180,  0.8775,  0.5869, -1.3853, -0.6356, -0.6922,  0.7399,  0.5402],
         [ 0.1798, -0.4656,  0.2638, -0.6801, -0.4169,  0.4765,  0.0991, -0.2992, -0.8500, -0.1792, -0.0935, -0.1088],
         [-0.8393,  0.6234, -0.7506, -0.4411, -0.1963, -0.0795,  0.0261,  0.1924, -0.3620,  1.1107, -0.1110,  0.4418],
         [-0.2403, -0.3683, -0.1956, -0.2543,  0.2528,  0.1956,  0.6195, -0.0164, -0.0104, -0.3045, -0.0634,  0.2639]]], grad_fn=<UnsafeViewBackward0>)


### QKV行列のQ,K,Vをヘッド数で分割
* 【2】QKVの行列を[Q1, Q2, K1, K2, V1, V2]と分割すれば良い

In [6]:
Q1, Q2, K1, K2, V1, V2 = QKV.view(1,5,3*H,D).transpose(1,2).unbind(dim=1)

In [7]:
# Qの部分が[Q1, Q2]となっていることが確認できます。残りも同様です。

print(f"Q1:\n{Q1}")
print(f"Q2:\n{Q2}")
print(f"K1:\n{K1}")

Q1:
tensor([[[-1.3839,  0.3560],
         [-0.3053, -0.4555],
         [ 0.1798, -0.4656],
         [-0.8393,  0.6234],
         [-0.2403, -0.3683]]], grad_fn=<UnbindBackward0>)
Q2:
tensor([[[-0.5477,  0.5145],
         [ 0.9167, -0.7092],
         [ 0.2638, -0.6801],
         [-0.7506, -0.4411],
         [-0.1956, -0.2543]]], grad_fn=<UnbindBackward0>)
K1:
tensor([[[ 1.5560, -0.1749],
         [ 0.2180,  0.8775],
         [-0.4169,  0.4765],
         [-0.1963, -0.0795],
         [ 0.2528,  0.1956]]], grad_fn=<UnbindBackward0>)


### ヘッド毎のattention weights

In [8]:
# softmax(Q1 K1 /√d)
head1_score = Q1@K1.transpose(-2,-1)/math.sqrt(D)
head1_attn_weights = torch.softmax(head1_score, dim=-1)
# softmax(Q1 K1 /√d) V1
head1 = head1_attn_weights@V1


# softmax(Q2 K2 /√d)
head2_score = Q2@K2.transpose(-2,-1)/math.sqrt(D)
head2_attn_weights = torch.softmax(head2_score, dim=-1)
# softmax(Q2 K2 /√d) V2
head2 = head2_attn_weights@V2

### mhaでの重みと個別に計算した重みの値が等しいことを目視

In [9]:
torch_w

tensor([[[[0.0424, 0.2048, 0.3446, 0.2414, 0.1667],
          [0.1729, 0.1644, 0.2146, 0.2448, 0.2033],
          [0.2667, 0.1591, 0.1675, 0.2068, 0.2000],
          [0.0698, 0.2457, 0.3001, 0.2061, 0.1782],
          [0.1792, 0.1710, 0.2114, 0.2354, 0.2030]],

         [[0.1456, 0.1290, 0.2313, 0.2845, 0.2096],
          [0.2896, 0.3155, 0.1334, 0.0994, 0.1622],
          [0.2136, 0.3166, 0.1714, 0.1335, 0.1649],
          [0.1254, 0.2581, 0.2383, 0.2125, 0.1655],
          [0.1764, 0.2372, 0.2087, 0.1930, 0.1846]]]], grad_fn=<ViewBackward0>)

In [10]:
head1_attn_weights, head2_attn_weights, 

(tensor([[[0.0424, 0.2048, 0.3446, 0.2414, 0.1667],
          [0.1729, 0.1644, 0.2146, 0.2448, 0.2033],
          [0.2667, 0.1591, 0.1675, 0.2068, 0.2000],
          [0.0698, 0.2457, 0.3001, 0.2061, 0.1782],
          [0.1792, 0.1710, 0.2114, 0.2354, 0.2030]]], grad_fn=<SoftmaxBackward0>),
 tensor([[[0.1456, 0.1290, 0.2313, 0.2845, 0.2096],
          [0.2896, 0.3155, 0.1334, 0.0994, 0.1622],
          [0.2136, 0.3166, 0.1714, 0.1335, 0.1649],
          [0.1254, 0.2581, 0.2383, 0.2125, 0.1655],
          [0.1764, 0.2372, 0.2087, 0.1930, 0.1846]]], grad_fn=<SoftmaxBackward0>))

In [11]:
attn_weights = torch.stack([head1_attn_weights, head2_attn_weights], dim=1)
print(torch.allclose(attn_weights, torch_w, atol=1e-6))  # True になるはず

True


### mhaの出力値と個別計算した値が等しいことの確認

In [12]:
head1 = head1_attn_weights@V1 # Attention(Q1,K1,V1)
head2 = head2_attn_weights@V2 # Attention(Q2,K2,V2)

concat_head = torch.concat([head1, head2], axis=2)
output = F.linear(concat_head, mha.out_proj.weight)

print(f"mhaの出力：\n{torch_output}")
print(f"個別計算の出力：\n{output}")

print(torch.allclose(output, torch_output , atol=1e-6))

mhaの出力：
tensor([[[ 0.3529,  0.0220,  0.0969, -0.1303],
         [ 0.4565,  0.1399,  0.0720,  0.1694],
         [ 0.3354,  0.1571,  0.0336,  0.2145],
         [ 0.3725,  0.0816,  0.1403, -0.0746],
         [ 0.3248,  0.0932,  0.0436,  0.0879]]], grad_fn=<TransposeBackward0>)
個別計算の出力：
tensor([[[ 0.3529,  0.0220,  0.0969, -0.1303],
         [ 0.4565,  0.1399,  0.0720,  0.1694],
         [ 0.3354,  0.1571,  0.0336,  0.2145],
         [ 0.3725,  0.0816,  0.1403, -0.0746],
         [ 0.3248,  0.0932,  0.0436,  0.0879]]], grad_fn=<UnsafeViewBackward0>)
True


## 論文風の実装
ベッド毎に行列を作成
* $Q_i = QW_i^Q$
* $K_i = KW_i^K$
* $V_i = VW_i^V$

$$
\text{head}_i = \text{softmax}(\frac{Q_i^Q K_i^T}{\sqrt{d}}) V_i \\
$$

$$
MultiHead(Q,K,V) = Concat(head_1,…,head_h) W^O
$$

**コードの流れ**
* ヘッドが2つなのでQ,K,V用の重みを(W_q1, W_k1, W_v1)と(W_q2, , W_k2,  W_v2)と2組準備する。
* 重み行列を利用してヘッド毎のQKVを(Q1, K1, V1)と(Q2, K2, V2)と2組計算する。
* Scaled Dot Product Attentionを計算。head1とhead2ができる
* [head1, head2]を並べて、出力用の重み$W^O$ (mha.out_proj.weight) を掛け算して最終出力

In [13]:
# Q, K, Vの作成
Q = X
K = X
V = X

# 小さな行列を先に作成
# mhaのWを利用してQ1などを作成する行列を作ります
W_q1, W_q2 = W_q[:2,:], W_q[2:,:]
W_k1, W_k2 = W_k[:2,:], W_k[2:,:]
W_v1, W_v2 = W_v[:2,:], W_v[2:,:]

# 個別にQiを作成
# ヘッド毎のQKV
Q1 = F.linear(Q, W_q1)    # Q1 = Q W_q1^T
Q2 = F.linear(Q, W_q2)
K1 = F.linear(K, W_k1)
K2 = F.linear(K, W_k2)
V1 = F.linear(V, W_v1)
V2 = F.linear(V, W_v2)

print(f"Q shape: {Q.shape}")
print(f"W_q1 shape: {W_q1.shape}")
print("ヘッド１用のクエリ行列")
print(f"Q1 shape: {Q1.shape}")

Q shape: torch.Size([1, 5, 4])
W_q1 shape: torch.Size([2, 4])
ヘッド１用のクエリ行列
Q1 shape: torch.Size([1, 5, 2])


* 各ヘッドについてScaled Dot-Product Attention を計算する。これが head_1, head_2
* concat(head_1, head_2)@W^Oによって最終出力とする。

In [14]:
# softmax(Q1 K1 /√d) V1の部分
head1_score = Q1@K1.transpose(-2,-1)/math.sqrt(D)
head1_attn_weights = torch.softmax(head1_score, dim=-1)
head1 = head1_attn_weights@V1

head2_score = Q2@K2.transpose(-2,-1)/math.sqrt(D)
head2_attn_weights = torch.softmax(head2_score, dim=-1)
head2 = head2_attn_weights@V2

In [15]:
concat_head = torch.concat([head1, head2], axis=2)
output = F.linear(concat_head, mha.out_proj.weight)
print(torch.allclose(output, torch_output , atol=1e-6))

True


In [16]:
# mhaの出力
torch_output

tensor([[[ 0.3529,  0.0220,  0.0969, -0.1303],
         [ 0.4565,  0.1399,  0.0720,  0.1694],
         [ 0.3354,  0.1571,  0.0336,  0.2145],
         [ 0.3725,  0.0816,  0.1403, -0.0746],
         [ 0.3248,  0.0932,  0.0436,  0.0879]]], grad_fn=<TransposeBackward0>)

In [17]:
# 順番に計算した出力
output

tensor([[[ 0.3529,  0.0220,  0.0969, -0.1303],
         [ 0.4565,  0.1399,  0.0720,  0.1694],
         [ 0.3354,  0.1571,  0.0336,  0.2145],
         [ 0.3725,  0.0816,  0.1403, -0.0746],
         [ 0.3248,  0.0932,  0.0436,  0.0879]]], grad_fn=<UnsafeViewBackward0>)